In [2]:
import pypinyin
import re
import numpy as np
import sys 

In [8]:
# 读入文件，经过处理之后写入test文件
with open("toutiao_cat_data.txt", "r", encoding="utf-8") as r:
    with open("test.txt","w",encoding="utf-8") as w:
        print("数据处理中：")
        data = r.readlines()
        lengthd=len(data)
        cnt=0
        last_cnt=cnt
        for line in data:
            s=[]
            for i in line:
                s.append("".join(re.findall(r'([\u4e00-\u9fa5]+)', i)))
            sentence=("".join(s))
            # print(sentence)
            pinyin_list = pypinyin.lazy_pinyin(sentence)
            pinyin=" ".join(pinyin_list)
            # print(pinyin)
            w.write(pinyin+"\n")
            w.write(sentence+"\n")
            cnt+=1
            if cnt-last_cnt>=10000 or cnt>lengthd-100:      #  运行状态，每隔一段时间才输出，减少输出占用的运行时间
                sys.stderr.write('\rEpoch: %d/%d' % (cnt,lengthd))  # 控制台动态输出err为红字，out为白字。
                sys.stderr.flush()
                last_cnt=cnt


数据处理中：


Epoch: 382688/382688

In [1]:
# 计算HMM模型参数
with open(r"test.txt", "r",encoding="utf-8") as txt:
    data = txt.read()
    txt.close()
    data = data.split('\n')

i = 0
last_i=i   # 标记运行状态
# 转移概率
A = {}
# 发射概率
B = {}
# 初始概率分布
pi = {}
# 次数就是数据集长度
train_time=len(data)
print("\n参数训练中：")
while (i != train_time):
    # 跳过空行
    if (data[i] == ""):
        i = i + 1
        continue
    py = data[i].strip().split()

    # 去除空格使句子与拼音下标对齐
    sentence = data[i + 1].strip()
    # print(py,sentence)
    # 统计汉字对应拼音的出现频率
    for j in range(len(sentence)):
        if sentence[j] not in B:
            B[sentence[j]] = {}
        if py[j] not in B[sentence[j]]:
            B[sentence[j]][py[j]] = 0
        B[sentence[j]][py[j]] += 1
    # 该行按空格分割成小句
    sentence = data[i + 1].strip().split()
    # print(sentence)
    for s in sentence:
        # 统计每句句首汉字的出现次数
        # print(s)
        if s[0] not in pi:
            pi[s[0]] = 0
        pi[s[0]] += 1
        # 对于相邻的两个字a和b，统计b在a后出现的次数
        for k in range(1, len(s)):
            if s[k - 1] not in A:
                A[s[k - 1]] = {}
            if s[k] not in A[s[k - 1]]:
                A[s[k - 1]][s[k]] = 0
            A[s[k - 1]][s[k]] += 1
    i = i + 2
    # if i > 12:
    #     break
    if i-last_i>=20000 or i>train_time-100:     # 每隔一段时间输出，减少输出占用的运行时间
        sys.stderr.write('\rEpoch: %d/%d' % (i,train_time))  # 控制台动态输出err为红字，out为白字。
        sys.stderr.flush()
        last_i=i

for i in A:
    temp=sum(A[i].values())
    # print(i, temp)
    for j in A[i]:
        A[i][j]=A[i][j]/temp
# print(A)

for i in B:
    temp = sum(B[i].values())
    for j in B[i]:
        B[i][j] = B[i][j] / temp
# print(B)

for i in pi:
    temp = sum(pi.values())
    pi[i] = pi[i] / temp
# print(pi)


参数训练中：


Epoch: 765376/765377

In [8]:
import time
# A,B,Π为模型参数，O为给定的观察序列
def Viterbi(pi, A, B, O):
    T = len(O)  # 隐状态长度
    # 定义δ和φ
    delta = {}  
    phi = {}
    delta_t0 = {}
    phi_t0 = {}
    start = time.perf_counter()
    # 初始化T=0时的delta和phi
    for i in pi:
        if i not in B or O[0] not in B[i]:
            B[i][O[0]] = 0
            continue
        delta_t0[i] = pi[i] * B[i][O[0]]
        phi_t0[i] = ""
    delta[0] = delta_t0
    phi[0] = phi_t0
    end = time.perf_counter()
    print("初始化时间：" + str(end - start))

    start = time.perf_counter()
    # 动态规划求解
    for t in range(1, T):
        temp_delta = {}
        temp_phi = {}
        # print(delta[t-1])
        for i in delta[t - 1]:
            if i not in A:
                continue
            for j in A[i]:
                # 判断不在的情况
                if j not in B or O[t] not in B[j]:
                    B[j][O[t]] = 0
                if j not in temp_delta:
                    temp_delta[j] = 0
                cal_now=delta[t - 1][i] * A[i][j] * B[j][O[t]]  # 当前概率
                if  cal_now> temp_delta[j]:     # 找最大值  
                    temp_delta[j] = cal_now
                    temp_phi[j] = i
    
        # 删除字典中概率为0的项以加速计算
        list = []
        for td in temp_delta:
            list.append(td)
        for l in list:
            if temp_delta[l] == 0:
                del temp_delta[l]

        delta[t] = temp_delta
        phi[t] = temp_phi

    end = time.perf_counter()
    print("动态规划时间：" + str(end - start))

    start = time.perf_counter()
    # 对求得的状态序列按照概率（键值对中的值）从小到大排序
    deltaT_sorted = sorted(delta[T - 1].items(), key=lambda x: x[1])
    # p为最大概率，state为其对应的状态
    p = deltaT_sorted[-1][1]    # 字典最后一个键值对的值
    end = time.perf_counter()
    print("排序时间：" + str(end - start))

    start = time.perf_counter()
    # 回溯得到最优状态
    state = []
    state.append(deltaT_sorted[-1][0])  # 字典最后一个键值对的键
    for t in range(T - 2, -1, -1):
        state.append(phi[t + 1][state[-1]])
    state = ''.join(reversed(state))
    end = time.perf_counter()
    print("回溯时间：" + str(end - start))
    return state

In [9]:
# 读入文件，经过处理之后写入test文件
with open(r"./测试集.txt", 'r') as r:       # 这个文件是gbk的
    test = r.read()
test = test.split('\n')  # 文本分割
i = 0
accuracy = []
print("\n开始测试：")

with open("mypredict.txt","w",encoding="utf-8") as w:
    while (i != len(test)):
        right = 0
        py = test[i].lower().split()
        print(py)
        predict = Viterbi(pi, A, B, py)
        truth = test[i + 1]
        # 保存预测结果
        w.write(predict + "\n")
        # 计算该句的正确率
        for j in range(len(predict)):
            if predict[j] == truth[j]:
                right += 1
        accuracy.append(right / len(predict))
        print(predict + " " + str(accuracy[-1] * 100.0) + "%")
        i = i + 2
        
print("平均正确率"+str(np.mean(accuracy)*100.0)+"%")


开始测试：
['jin', 'tian', 'wan', 'shang', 'you', 'hao', 'kan', 'de', 'dian', 'ying']
初始化时间：0.003612599999996746
动态规划时间：7.85133529999996
排序时间：0.0011951000000181011
回溯时间：1.1799999981576548e-05
今天晚上有好看的电影 100.0%
['bei', 'jing', 'ao', 'yun', 'hui', 'kai', 'mu', 'shi', 'fei', 'chang', 'jing', 'cai']
初始化时间：0.0026737000000025546
动态规划时间：9.459739599999978
排序时间：0.04407069999996338
回溯时间：1.2200000014672696e-05
北京奥运会开幕是非常精彩 91.66666666666666%
['quan', 'guo', 'ren', 'min', 'dai', 'biao', 'da', 'hui', 'zai', 'bei', 'jing', 'ren', 'min', 'da', 'hui', 'tang', 'long', 'zhong', 'zhao', 'kai']
初始化时间：0.0029197999999723834


KeyboardInterrupt: 